# NLP를 활용한 레시피 추천 챗봇 구현
**개요**
- Collection 
- Preprocessing
- EDA
- Embeddings
- Modeling
- Streamlit

## 2. Preprocessing (전처리)
- 수집한 데이터 : 요리이름, 카테고리, 난이도, 소요시간, 재료, 요리방법, 요리설명, 링크, 이미지링크
- 전처리할 데이터
    - 재료에 섞인 불용어 위주로 제거
    - 설명에 '비만', '당뇨'가 들어가는 음식 체크


### 라이브러리 import

In [1]:
# 데이터 분석
import numpy as np
import pandas as pd

# 진행시간 표시
import time
from tqdm.notebook import tqdm
tqdm.pandas()

# 정규표현식
import re

### 데이터 불러오기

In [2]:
# 크롤링한 데이터 불러오기
raw_recipes = pd.read_csv('data/raw_recipes.csv')

In [3]:
raw_recipes.shape

(4947, 9)

In [4]:
raw_recipes.head(1)

,요리,종류,난이도,소요시간,링크,사진,재료,요리방법,설명
0,올리브 소시지 솥밥,메인요리,1,30,https://wtable.co.kr/recipes/rF7F5gmLrySsw7ZXZ...,https://static.wtable.co.kr/image/production/s...,"['쌀', '물', '치킨스톡 파우더', '샬롯(또는 양파)', '그린올리브', '...","['쌀은 씻어 채에 받쳐 30분 정도 불려주세요.\xa0', '소세지는 모양대로 슬...",올리브를 넣어 이국적인 감칠맛이 매력적인 솥 밥. 별다른 양념없이도 술술 넘어가는 ...


### 전처리 진행하기: 재료

In [5]:
# 수기로 발견한 대표적인 불용어와 많은 레시피에 두루 포함되는 조미료 계열을 리스트 형태로 변수 할당
stop_words = ['송송','썬','삶은','짤주머니','사각','X','곱게','리덕션','채썬',
              '냉장','늙은','판','미니','집','흰','두절','레드','모양','같이','잘','익은',
              '고운','익힌','주머니','건세모가사리','빨강','썰은','소분한','작은',
              '마른','벗긴','밥심','뜨거운','따듯한','씨를','크기로','제거','생물','빨간',
              '갈은','지은','데친','국물용','부순','꼬치','찬','차가운','빨간','중하',
              '컬러','구멍','따뜻한','A','틀','초밥용','몽글몽귤','튜브','두꺼운','크기','모둠',
              '해감한','와','구운','굵은','그린','굵게','익은','시판','것','없는','cm','상온',
              '삶은','무첨가','이지잇','잘게', '자른','깐','찐','씻은','잡채용',
              '고운', '면수']

In [6]:
# 텍스트 전처리 (1차)
def preprocessing(text):
    '''
    문자열을 전달하면 불용어, 중복, 공백을 제거하여 리스트 형태로 반환하는 함수

    Args:
    - 문자열 (예 : "['순살 고등어', '청주(또는 맛술)', '소금', '소금']")

    Returns:
    - 리스트 (예 : ['순살 고등어', '청주', '소금'])

    Example:
    >>> preprocessing("['순살 고등어', '청주(또는 맛술)', '소금', '소금']")
    ['순살고등어', '청주', '소금']
    '''
    process = eval(text)

    process = [elem for elem in process if not re.search(r'\*', elem)]
    process = [elem.replace('\xa0', '') for elem in process]
    process = [elem.replace(' ', '') for elem in process]

    process = ' '.join(set(process))
    process = re.sub(r'\([^)]*\)', '', process).strip()
    process = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', process).strip()
    pattern = '|'.join(stop_words)
    process = re.sub(pattern, '', process)
    process = re.sub(r'또는', ' ', process)

    process = process.split(' ')

    process = [elem for elem in process if elem != '']
    
    return process

In [7]:
# 데이터 프레임 복사
df = raw_recipes.copy()

In [8]:
# 재료 컬럼에 텍스트 전처리 (1차) 적용
df['재료'] = df['재료'].progress_map(preprocessing)
df['재료']

  0%|          | 0/4947 [00:00<?, ?it/s]

0       [소시지, 쌀, 올리브오일, 치킨스톡파우더, 물, 후춧가루, 파마산치즈, 올리브, ...
1        [파스타면, 소금, 올리브오일, 물, 후춧가루, 올리브, 파슬리, 방울토마토, 케이퍼]
2            [브리치즈, 올리브오일, 마늘, 바게트, 꿀, 로즈마리, 후춧가루, 방울토마토]
3       [소금, 루꼴라, 복숭아, 올리브오일, 화이트발사믹, 민트잎, 부라타치즈, 꿀, 후...
4        [소금, 양파, 고수, 아보카도, 올리브오일, 토마토, 슈파마산치즈, 토르티야, 레몬]
                              ...                        
4942         [설탕, 골뱅이통조림, 튀김가루, 물, 간장, 식초, 고춧가루, 식용유, 대파]
4943          [다진파슬리, 다진마늘, 청주, 대하, 피자치즈, 후춧가루, 마요네즈, 레몬]
4944    [소금, 올리브오일, 새송이버섯, 아스파라거스, 녹인버터, 로즈마리, 타임, 후춧가...
4945                        [다진파슬리, 달걀, 피자치즈, 단호박, 슬라이스햄]
4946    [소금, 두유, 타임, 대파, 디종머스타드, 코코넛밀크, 후춧가루, 뉴트리셔널이스트...
Name: 재료, Length: 4947, dtype: object

In [9]:
# 확인하기
df.head(2)

,요리,종류,난이도,소요시간,링크,사진,재료,요리방법,설명
0,올리브 소시지 솥밥,메인요리,1,30,https://wtable.co.kr/recipes/rF7F5gmLrySsw7ZXZ...,https://static.wtable.co.kr/image/production/s...,"[소시지, 쌀, 올리브오일, 치킨스톡파우더, 물, 후춧가루, 파마산치즈, 올리브, ...","['쌀은 씻어 채에 받쳐 30분 정도 불려주세요.\xa0', '소세지는 모양대로 슬...",올리브를 넣어 이국적인 감칠맛이 매력적인 솥 밥. 별다른 양념없이도 술술 넘어가는 ...
1,원 팟 파스타,메인요리,1,20,https://wtable.co.kr/recipes/EzDURnKruCpR2nGxZ...,https://static.wtable.co.kr/image/production/s...,"[파스타면, 소금, 올리브오일, 물, 후춧가루, 올리브, 파슬리, 방울토마토, 케이퍼]","['방울토마토는 반으로 썰어주세요.\xa0파슬리는 다져주세요.', '냄비에 파스타면...","냄비에 파스타와 토마토, 올리브 등 재료를 넣고 물을 부은 후 끓이면 완성되는 초간..."


In [10]:
# 재료수 파생변수를 생성하는 함수
def ingredients_count(text):
    ing_count = len(text)
    return ing_count

In [11]:
# 재료 컬럼에 재료수 파생변수 생성
df['재료수'] = df['재료'].progress_map(ingredients_count)
df.head(2)

  0%|          | 0/4947 [00:00<?, ?it/s]

,요리,종류,난이도,소요시간,링크,사진,재료,요리방법,설명,재료수
0,올리브 소시지 솥밥,메인요리,1,30,https://wtable.co.kr/recipes/rF7F5gmLrySsw7ZXZ...,https://static.wtable.co.kr/image/production/s...,"[소시지, 쌀, 올리브오일, 치킨스톡파우더, 물, 후춧가루, 파마산치즈, 올리브, ...","['쌀은 씻어 채에 받쳐 30분 정도 불려주세요.\xa0', '소세지는 모양대로 슬...",올리브를 넣어 이국적인 감칠맛이 매력적인 솥 밥. 별다른 양념없이도 술술 넘어가는 ...,10
1,원 팟 파스타,메인요리,1,20,https://wtable.co.kr/recipes/EzDURnKruCpR2nGxZ...,https://static.wtable.co.kr/image/production/s...,"[파스타면, 소금, 올리브오일, 물, 후춧가루, 올리브, 파슬리, 방울토마토, 케이퍼]","['방울토마토는 반으로 썰어주세요.\xa0파슬리는 다져주세요.', '냄비에 파스타면...","냄비에 파스타와 토마토, 올리브 등 재료를 넣고 물을 부은 후 끓이면 완성되는 초간...",9


In [12]:
# 텍스트 전처리 (2차)
def preprocessing2(list):
    '''
    리스트를 전달하면 공백으로 구분된 하나의 문자열로 반환하는 함수

    Args:
    - 리스트 (예 : ['순살 고등어', '청주', '소금'])

    Returns:
    - 문자열 (예 : "순살고등어 청주 소금")

    Example:
    >>> preprocessing2(['순살 고등어', '청주', '소금'])
    "순살고등어 청주 소금"
    '''
    process = str(list)
    process = eval(process)
    process = ' '.join(set(process))

    return process

In [13]:
# 재료 컬럼에 텍스트 전처리 (2차) 적용
df['재료'] = df['재료'].progress_map(preprocessing2)
df.head(2)

  0%|          | 0/4947 [00:00<?, ?it/s]

,요리,종류,난이도,소요시간,링크,사진,재료,요리방법,설명,재료수
0,올리브 소시지 솥밥,메인요리,1,30,https://wtable.co.kr/recipes/rF7F5gmLrySsw7ZXZ...,https://static.wtable.co.kr/image/production/s...,소시지 올리브오일 쌀 치킨스톡파우더 물 후춧가루 파마산치즈 파슬리 올리브 샬롯,"['쌀은 씻어 채에 받쳐 30분 정도 불려주세요.\xa0', '소세지는 모양대로 슬...",올리브를 넣어 이국적인 감칠맛이 매력적인 솥 밥. 별다른 양념없이도 술술 넘어가는 ...,10
1,원 팟 파스타,메인요리,1,20,https://wtable.co.kr/recipes/EzDURnKruCpR2nGxZ...,https://static.wtable.co.kr/image/production/s...,파스타면 소금 올리브오일 물 후춧가루 파슬리 올리브 방울토마토 케이퍼,"['방울토마토는 반으로 썰어주세요.\xa0파슬리는 다져주세요.', '냄비에 파스타면...","냄비에 파스타와 토마토, 올리브 등 재료를 넣고 물을 부은 후 끓이면 완성되는 초간...",9


### 전처리 진행하기: 설명

In [14]:
# 불용어(조사) 리스트
stopwords = ['는', '은', '이', '에', '의', '가', '이', '도','을', '를', '으로', '에서', '와', '과', '한', '하고', '을', '로', '에', '나', '다가', '하게', '해서', '들', '고', '도', '에도']

In [15]:
def preprocess_description(description, stopwords):
    # 한글만 남기기
    words = re.sub(r'[^가-힣]', ' ', description)
    # 설명을 띄어쓰기 단위로 나누기
    words = words.split()
    # 각 단어에서 맨 뒤에 붙어있는 불용어 제거
    cleaned_words = []
    for word in words:
        for stopword in stopwords:
            if word.endswith(stopword):
                word = word[:-len(stopword)]
        cleaned_words.append(word)
    
    return cleaned_words

In [16]:
# 전처리된 설명 컬럼 추가
df['전처리된_설명'] = df['설명'].apply(lambda x: preprocess_description(x, stopwords))

In [17]:
df.head(5)

,요리,종류,난이도,소요시간,링크,사진,재료,요리방법,설명,재료수,전처리된_설명
0,올리브 소시지 솥밥,메인요리,1,30,https://wtable.co.kr/recipes/rF7F5gmLrySsw7ZXZ...,https://static.wtable.co.kr/image/production/s...,소시지 올리브오일 쌀 치킨스톡파우더 물 후춧가루 파마산치즈 파슬리 올리브 샬롯,"['쌀은 씻어 채에 받쳐 30분 정도 불려주세요.\xa0', '소세지는 모양대로 슬...",올리브를 넣어 이국적인 감칠맛이 매력적인 솥 밥. 별다른 양념없이도 술술 넘어가는 ...,10,"[올리브, 넣어, 이국적인, 감칠맛, 매력적인, 솥, 밥, 별다른, 양념없이, 술술..."
1,원 팟 파스타,메인요리,1,20,https://wtable.co.kr/recipes/EzDURnKruCpR2nGxZ...,https://static.wtable.co.kr/image/production/s...,파스타면 소금 올리브오일 물 후춧가루 파슬리 올리브 방울토마토 케이퍼,"['방울토마토는 반으로 썰어주세요.\xa0파슬리는 다져주세요.', '냄비에 파스타면...","냄비에 파스타와 토마토, 올리브 등 재료를 넣고 물을 부은 후 끓이면 완성되는 초간...",9,"[냄비, 파스타, 토마토, 올리브, 등, 재료, 넣, 물, 부, 후, 끓이면, 완성..."
2,그릴드 브리치즈,메인요리,1,30,https://wtable.co.kr/recipes/Tzr7zXZuJvm35s1wd...,https://static.wtable.co.kr/image/production/s...,브리치즈 마늘 올리브오일 바게트 꿀 로즈마리 후춧가루 방울토마토,"['브리치즈에 칼집을 넣어주세요. 마늘은 편을 썰어주세요.\xa0', '오븐용기에 ...","브리치즈와 토마토, 오일과 허브를 뿌린 후 오븐에 구워내 보세요. 와인안주로 활용하...",8,"[브리치즈, 토마토, 오일, 허브, 뿌린, 후, 오븐, 구워내, 보세요, 와인안주,..."
3,그릴드 피치 샐러드,메인요리,1,25,https://wtable.co.kr/recipes/RofraF8aFP5u5rqeX...,https://static.wtable.co.kr/image/production/s...,소금 루꼴라 복숭아 올리브오일 화이트발사믹 민트잎 꿀 후춧가루 방울토마토 부라타치즈,['복숭아는 씨를 빼고 웨지 모양으로 썰어주세요. 작은 것은 반으로 썰어 주세요.\...,복숭아에 멋진 그릴 자국을 내보세요. 여름을 맞아 제철인 단단한 복숭아를 그릴에 구...,10,"[복숭아, 멋진, 그릴, 자국, 내보세요, 여름, 맞아, 제철인, 단단, 복숭아, ..."
4,과카몰리 부리또콘,메인요리,1,20,https://wtable.co.kr/recipes/3PW95soZHRArnzsYB...,https://static.wtable.co.kr/image/production/s...,소금 양파 올리브오일 아보카도 고수 토마토 토르티야 슈파마산치즈 레몬,['또띠아를 반으로 자른 후 원 뿔 모양으로 둥글게 말아 나무 꼬치로 가장자리를 고...,아보카도는 불포화 지방산이 풍부해 콜레스테롤 수치를 낮추고 혈관건강에 유익해요. 게...,9,"[아보카, 불포화, 지방산, 풍부해, 콜레스테롤, 수치, 낮추, 혈관건강, 유익해요..."


In [18]:
# 질병 유무 판단
def extract_disease(words):
    # 질병 유무 판단
    if any(keyword in words for keyword in ['비만', '다이어트', '비만예방']):
        return '비만'
    elif any(keyword in words for keyword in ['당뇨', '당뇨병', '혈당']):
        return '당뇨'
    else:
        return '일반'

In [19]:
df['질병'] = df['전처리된_설명'].apply(extract_disease)

In [20]:
df.head(5)

,요리,종류,난이도,소요시간,링크,사진,재료,요리방법,설명,재료수,전처리된_설명,질병
0,올리브 소시지 솥밥,메인요리,1,30,https://wtable.co.kr/recipes/rF7F5gmLrySsw7ZXZ...,https://static.wtable.co.kr/image/production/s...,소시지 올리브오일 쌀 치킨스톡파우더 물 후춧가루 파마산치즈 파슬리 올리브 샬롯,"['쌀은 씻어 채에 받쳐 30분 정도 불려주세요.\xa0', '소세지는 모양대로 슬...",올리브를 넣어 이국적인 감칠맛이 매력적인 솥 밥. 별다른 양념없이도 술술 넘어가는 ...,10,"[올리브, 넣어, 이국적인, 감칠맛, 매력적인, 솥, 밥, 별다른, 양념없이, 술술...",일반
1,원 팟 파스타,메인요리,1,20,https://wtable.co.kr/recipes/EzDURnKruCpR2nGxZ...,https://static.wtable.co.kr/image/production/s...,파스타면 소금 올리브오일 물 후춧가루 파슬리 올리브 방울토마토 케이퍼,"['방울토마토는 반으로 썰어주세요.\xa0파슬리는 다져주세요.', '냄비에 파스타면...","냄비에 파스타와 토마토, 올리브 등 재료를 넣고 물을 부은 후 끓이면 완성되는 초간...",9,"[냄비, 파스타, 토마토, 올리브, 등, 재료, 넣, 물, 부, 후, 끓이면, 완성...",일반
2,그릴드 브리치즈,메인요리,1,30,https://wtable.co.kr/recipes/Tzr7zXZuJvm35s1wd...,https://static.wtable.co.kr/image/production/s...,브리치즈 마늘 올리브오일 바게트 꿀 로즈마리 후춧가루 방울토마토,"['브리치즈에 칼집을 넣어주세요. 마늘은 편을 썰어주세요.\xa0', '오븐용기에 ...","브리치즈와 토마토, 오일과 허브를 뿌린 후 오븐에 구워내 보세요. 와인안주로 활용하...",8,"[브리치즈, 토마토, 오일, 허브, 뿌린, 후, 오븐, 구워내, 보세요, 와인안주,...",일반
3,그릴드 피치 샐러드,메인요리,1,25,https://wtable.co.kr/recipes/RofraF8aFP5u5rqeX...,https://static.wtable.co.kr/image/production/s...,소금 루꼴라 복숭아 올리브오일 화이트발사믹 민트잎 꿀 후춧가루 방울토마토 부라타치즈,['복숭아는 씨를 빼고 웨지 모양으로 썰어주세요. 작은 것은 반으로 썰어 주세요.\...,복숭아에 멋진 그릴 자국을 내보세요. 여름을 맞아 제철인 단단한 복숭아를 그릴에 구...,10,"[복숭아, 멋진, 그릴, 자국, 내보세요, 여름, 맞아, 제철인, 단단, 복숭아, ...",일반
4,과카몰리 부리또콘,메인요리,1,20,https://wtable.co.kr/recipes/3PW95soZHRArnzsYB...,https://static.wtable.co.kr/image/production/s...,소금 양파 올리브오일 아보카도 고수 토마토 토르티야 슈파마산치즈 레몬,['또띠아를 반으로 자른 후 원 뿔 모양으로 둥글게 말아 나무 꼬치로 가장자리를 고...,아보카도는 불포화 지방산이 풍부해 콜레스테롤 수치를 낮추고 혈관건강에 유익해요. 게...,9,"[아보카, 불포화, 지방산, 풍부해, 콜레스테롤, 수치, 낮추, 혈관건강, 유익해요...",일반


In [21]:
# '전처리된_설명' 컬럼 제거
df.drop(columns=['전처리된_설명'], inplace=True)

In [22]:
df.head(10)

,요리,종류,난이도,소요시간,링크,사진,재료,요리방법,설명,재료수,질병
0,올리브 소시지 솥밥,메인요리,1,30,https://wtable.co.kr/recipes/rF7F5gmLrySsw7ZXZ...,https://static.wtable.co.kr/image/production/s...,소시지 올리브오일 쌀 치킨스톡파우더 물 후춧가루 파마산치즈 파슬리 올리브 샬롯,"['쌀은 씻어 채에 받쳐 30분 정도 불려주세요.\xa0', '소세지는 모양대로 슬...",올리브를 넣어 이국적인 감칠맛이 매력적인 솥 밥. 별다른 양념없이도 술술 넘어가는 ...,10,일반
1,원 팟 파스타,메인요리,1,20,https://wtable.co.kr/recipes/EzDURnKruCpR2nGxZ...,https://static.wtable.co.kr/image/production/s...,파스타면 소금 올리브오일 물 후춧가루 파슬리 올리브 방울토마토 케이퍼,"['방울토마토는 반으로 썰어주세요.\xa0파슬리는 다져주세요.', '냄비에 파스타면...","냄비에 파스타와 토마토, 올리브 등 재료를 넣고 물을 부은 후 끓이면 완성되는 초간...",9,일반
2,그릴드 브리치즈,메인요리,1,30,https://wtable.co.kr/recipes/Tzr7zXZuJvm35s1wd...,https://static.wtable.co.kr/image/production/s...,브리치즈 마늘 올리브오일 바게트 꿀 로즈마리 후춧가루 방울토마토,"['브리치즈에 칼집을 넣어주세요. 마늘은 편을 썰어주세요.\xa0', '오븐용기에 ...","브리치즈와 토마토, 오일과 허브를 뿌린 후 오븐에 구워내 보세요. 와인안주로 활용하...",8,일반
3,그릴드 피치 샐러드,메인요리,1,25,https://wtable.co.kr/recipes/RofraF8aFP5u5rqeX...,https://static.wtable.co.kr/image/production/s...,소금 루꼴라 복숭아 올리브오일 화이트발사믹 민트잎 꿀 후춧가루 방울토마토 부라타치즈,['복숭아는 씨를 빼고 웨지 모양으로 썰어주세요. 작은 것은 반으로 썰어 주세요.\...,복숭아에 멋진 그릴 자국을 내보세요. 여름을 맞아 제철인 단단한 복숭아를 그릴에 구...,10,일반
4,과카몰리 부리또콘,메인요리,1,20,https://wtable.co.kr/recipes/3PW95soZHRArnzsYB...,https://static.wtable.co.kr/image/production/s...,소금 양파 올리브오일 아보카도 고수 토마토 토르티야 슈파마산치즈 레몬,['또띠아를 반으로 자른 후 원 뿔 모양으로 둥글게 말아 나무 꼬치로 가장자리를 고...,아보카도는 불포화 지방산이 풍부해 콜레스테롤 수치를 낮추고 혈관건강에 유익해요. 게...,9,일반
5,아스파라거스 부라타 샐러드,메인요리,1,10,https://wtable.co.kr/recipes/EZjwzNqQdGsDLKhPM...,https://static.wtable.co.kr/image/production/s...,소금 후추 올리브오일 마늘 아스파라거스 레몬 부라타치즈,['아스파라거스는 아래 2cm 정도 밑둥을 잘라 내고 채칼로 얇게 썰어주세요. 마늘...,보기만해도 건강해 질것 같은 초록빛 아스파라거스는 아삭한 식감에 영양소가 풍부해 많...,7,일반
6,베지케이크,메인요리,2,60,https://wtable.co.kr/recipes/LX9nc5kixS9BhwjC1...,https://static.wtable.co.kr/image/production/s...,소금 후추 패스츄리생지 생크림 달걀 올리브오일 마늘 토마토 파슬리 자투리채소,"['주키니, 당근, 감자, 양파, 토마토는 슬라이서로 얇게 썰어주세요.', '볼에 ...",요즘 SNS상에서 많이 만들어 화제가 되고있는 #크링클 케이크 아시나요~? 얇은 페...,10,일반
7,매실청,메인요리,1,30,https://wtable.co.kr/recipes/kbe6SgVjggshTGGsh...,https://static.wtable.co.kr/image/production/s...,청매실 설탕,['매실을 흐르는 물에 세척 후 채반에 받쳐주세요. 키친타올에 올려 물기를 제거해 ...,매실을 설탕에 절여 발효시킨 매실청! 에이드로 마시거나 다양한 요리에 사용되어 거의...,2,일반
8,연어 크림치즈 찹 베이글,메인요리,1,10,https://wtable.co.kr/recipes/kY6F6aVJKwnQ6fBrp...,https://static.wtable.co.kr/image/production/s...,소금 레몬제스트 샐러리 딜 크림치즈 훈제연어 레몬즙 올리고당 후춧가루 베이글,"['도마에 모든 재료를 올린 후 칼로 다져주세요. \r\n', '베이글에 연어 크림...","고소한 크림치즈와 훈제연어, 샐러리, 딜, 레몬 등 좋아하는 재료를 도마에 올려 잘...",10,일반
9,히포크라테스 수프 (야채수프),메인요리,1,120,https://wtable.co.kr/recipes/8nrMEFd4P3gbJ8BF2...,https://static.wtable.co.kr/image/production/s...,양파 다진파슬리 마늘 완숙토마토 샐러리 감자 대파,"['깨끗이 씻은 토마토, 감자, 양파, 대파, 샐러리는 깍둑 썰어주세요. \r\n(...","수천 년 전, 히포크라테스는 '음식이 네 약이 되게 하라 고 말했습니다. 그의 지혜...",7,일반


In [23]:
df[df['질병'] == '비만']

,요리,종류,난이도,소요시간,링크,사진,재료,요리방법,설명,재료수,질병
42,양배추 대패삼겹살찜,메인요리,1,20,https://wtable.co.kr/recipes/DFQuwmKU3MfHU8etn...,https://static.wtable.co.kr/image/production/s...,소금 설탕 청주 양배추 후춧가루 레몬즙 쪽파 쯔유 통깨 대패삼겹살,['양배추는 식초를 탄 물에 10분 정도 담가 두었다가 흐르는 물에 깨끗이 씻어주세...,양배추와 대패삼겹살을 쯔유 소스와 함께 쪄 내면 간단하고 맛있는 다이어트 요리가 탄...,10,비만
56,메밀쌀 샐러드,메인요리,1,35,https://wtable.co.kr/recipes/4PyU6iSvnLc2AdvAB...,https://static.wtable.co.kr/image/production/s...,샐러드채소 소금 적양배추 올리브오일 후춧가루 올리고당 레몬즙 마요네즈 오이 요거트 ...,['끓는 물에 소금과 메밀쌀을 넣고 15~20분 정도 삶아주세요. 찬물에 헹궈 물기...,샐러드를 먹을 때 식감이 조금 아쉬울 때가 있잖아요. 메밀쌀을 넣으면 구수한 맛을 ...,12,비만
68,갈릭버터쉬림프 포케,메인요리,1,30,https://wtable.co.kr/recipes/FTfisrPGbUiBZ9goK...,https://static.wtable.co.kr/image/production/s...,적양파 소금 올리브유 맛술 베이컨 마늘 아보카도 새우 보리귀리밥 후춧가루 페퍼론치노...,['새우는 맛술을 뿌려 10분 정도 재웠다가 키친타월에 올려 물기를 제거해 주세요....,다이어트 하거나 탄단지가 균형 잡힌 식사를 원할 때 포케를 많이 찾게 되잖아요. 이...,16,비만
74,땅콩잼,메인요리,1,5,https://wtable.co.kr/recipes/TSbQxxADfGwGitMAL...,https://static.wtable.co.kr/image/production/s...,볶은땅콩,"['볶은 땅콩은 껍질을 벗긴 후 믹서에 넣고 갈아 주세요. ', '완성된 땅콩잼을 ...",당분이나 오일 없이 오직 땅콩만 곱게 갈아 땅콩잼을 만들어 보세요. 믹서에 땅콩을 ...,1,비만
174,스리라차 치킨과 밀크셰이크,메인요리,2,30,https://wtable.co.kr/recipes/9hfhaE9JubbJUkR7N...,https://static.wtable.co.kr/image/production/s...,소금 설탕 올리브유 얼음 다진마늘 물 후춧가루 튀김가루 레몬즙 닭다리살 식초 마요네...,['닭다리살은 우유에 20분 가량 담갔다가 물에 씻어 물기를 제거해 주세요. 닭다리...,스리라차 소스는 매운 고추가 들어가는 태국식 칠리소스인데요. 칼로리가 낮아 다이어트...,17,비만
...,...,...,...,...,...,...,...,...,...,...,...
4725,양배추 샐러드,키토,1,10,https://wtable.co.kr/recipes/5W1fWqh5VMtB6ajEE...,https://static.wtable.co.kr/image/production/s...,소금 올리브오일 디종머스타드 아가베시럽 후춧가루 식초 케일 잎채소 당근 양배추,['깨끗이\xa0씻은\xa0양배추는\xa0두꺼운\xa0심지\xa0부분을\xa0자르고...,"채소를 통해 몸과 마음을 더욱더 이롭게, 나아가 자연과 함께 살아가는 건강한 삶을...",10,비만
4735,아마란스 샐러드,키토,1,25,https://wtable.co.kr/recipes/3imkdCpe99wCqffFa...,https://static.wtable.co.kr/image/production/s...,소금 후추 발사믹식초 올리브오일 느타리버섯 토마토 새송이버섯 아마란스 적채 올리고당...,"['아마란스는 끓는 물에 넣어 10분간 삶은 후 물기를 제거해주세요.', '토마토는...","슈퍼푸드 중 하나로 손꼽히는 아마란스를 샐러드로 즐겨보세요. 단백질 함량이 높고, ...",11,비만
4738,버섯샐러드,키토,1,25,https://wtable.co.kr/recipes/tFxi5bu3sDpBg8EXu...,https://static.wtable.co.kr/image/production/s...,소금 양파 후추 발사믹식초 올리브오일 느타리버섯 만가닥버섯 새송이버섯 올리고당 프랑...,['미니 새송이버섯은 길게 이등분하고 느타리버섯과 만가닥버섯은 밑동을 자르고 먹기 ...,버섯은 고유의 맛과 향이 뛰어나며 다양하게 조리할 수 있어 세계 어디에서나 사랑 받...,11,비만
4888,낫또 아보카도 덮밥,오븐 요리,3,600,https://wtable.co.kr/recipes/eqFzFufFecb83RSxA...,https://static.wtable.co.kr/image/production/s...,소금 두부 아보카도 연겨자 식용유 후춧가루 고추냉이 간장 쯔유 불린메주콩 참기름 낫...,"['불린 콩을 밥솥에 넣고 물 ½ 컵을 넣어 잡곡 취사 모드로 삶아주세요.', '삶...",식이섬유와 단백질이 풍부한 낫또와 숲의 버터라고 불리는 고소한 아보카도는 맛도 있지...,15,비만


In [24]:
# csv 파일로 저장하고 불러오기
df.to_csv('data/preprocessed_recipes.csv', index=False)
n_df = pd.read_csv('data/preprocessed_recipes.csv')

In [25]:
# 불용어 제거 확인하기 
filtered_df = n_df[n_df['재료'].apply(lambda x: any(word in x for word in stop_words))]
filtered_df 

,요리,종류,난이도,소요시간,링크,사진,재료,요리방법,설명,재료수,질병


In [26]:
# 텍스트 전처리 전과 후를 비교하는 함수
def sample_test(origin_df, processed_df, column_name):
    sample = origin_df.sample(1)[column_name]
    processed_sample = processed_df.iloc[list(sample.index)][column_name]
    with pd.option_context('display.max_colwidth', None):
        return display(sample.to_string(index=False), processed_sample.to_string(index=False))

In [27]:
# 실행할 때 마다 sample 무작위로 변경하면서 확인 가능
sample_test(raw_recipes, df, '재료')

"['소고기 사태', '물', '팽이버섯', '부추', '알배추', '대파', '통후추', '마늘', '국간장', '소금', '간장', '물', '설탕', '식초', '다진 파', '다진 청양고추', '연겨자']"

'소금 설탕 다진청양고추 통후추 알배추 마늘 국간장 팽이버섯 소고기사태 연겨자 물 간장 식초 다진파 대파 부추'